# Dubai License Plate Detection

This notebook demonstrates different approaches for detecting and recognizing Dubai license plates using YOLOv9.

In [3]:
import os
import sys
import cv2
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, clear_output

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 8]  # Set default figure size

# Add the yolov9 directory to Python path
# This is crucial for importing the YOLOv9 modules correctly
yolov9_path = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "yolov9"))
if not os.path.exists(yolov9_path):
    yolov9_path = os.path.abspath(os.path.join(os.getcwd(), "yolov9"))

if yolov9_path not in sys.path:
    sys.path.insert(0, yolov9_path)
    print(f"Added YOLOv9 path: {yolov9_path}")

# Now we can import YOLOv9 modules
from yolov9.models.common import DetectMultiBackend
from yolov9.utils.dataloaders import LoadImages
from yolov9.utils.general import non_max_suppression, scale_boxes
from yolov9.utils.plots import Annotator, colors

# Add the src directory to Python path if it's not already there
src_path = os.path.abspath(os.path.dirname(os.getcwd()))
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"Added src path: {src_path}")

Added YOLOv9 path: /Users/zg0ul/Coding/SAGER/dubai_license/src/yolov9
Added src path: /Users/zg0ul/Coding/SAGER/dubai_license


In [4]:
# Define the extract_license_info function
def extract_license_info(detections):
    """Extract emirate, category, and license number from license plate detections"""
    if not detections:
        return {'emirate': 'Unknown', 'category': 'Unknown', 'plate_number': 'Unknown'}

    # Convert detections to a more convenient format
    elements = []
    for detection in detections:
        if isinstance(detection, dict) and 'bbox' in detection:
            # If detection is already in dictionary format
            x1, y1, x2, y2 = detection['bbox']
            cls = detection['class']
            conf = detection['confidence']
        else:
            # If detection is in the format from your output
            cls = detection[0]
            conf = detection[1]
            x1, y1, x2, y2 = map(int, detection[2].strip('()').split(','))

        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1
        area = width * height

        elements.append({
            'class': cls,
            'confidence': conf,
            'bbox': (x1, y1, x2, y2),
            'center': (center_x, center_y),
            'width': width,
            'height': height,
            'area': area
        })

    # First identify the plate itself if it's in the detections
    plate_elements = [e for e in elements if 'plate' in str(e['class']).lower()]
    plate_bbox = None
    if plate_elements:
        # Use the largest plate detection
        plate_elements.sort(key=lambda x: x['area'], reverse=True)
        plate_bbox = plate_elements[0]['bbox']
        # Remove plate from further processing
        elements = [e for e in elements if e not in plate_elements]

    # Identify emirate - typically contains "DUBAI" in the class name
    emirate_elements = [e for e in elements if any(em in str(e['class']).upper() 
                        for em in ['DUBAI', 'AJMAN', 'SHARKA', 'ABUDABI', 'FUJIRA', 'RAK', 'AM'])]
    emirate = emirate_elements[0]['class'] if emirate_elements else 'Unknown'

    # Remove emirate from further processing
    if emirate_elements:
        for e in emirate_elements:
            if e in elements:  # Check if element is still in the list
                elements.remove(e)

    # If no elements left after removing emirate, return early
    if not elements:
        return {
            'emirate': emirate,
            'category': 'Unknown',
            'plate_number': 'Unknown'
        }

    # Group elements by their vertical position (y-coordinate)
    y_values = [e['center'][1] for e in elements]
    min_y, max_y = min(y_values), max(y_values)
    vertical_range = max_y - min_y

    # Check for known category letters common in UAE plates
    known_categories = set(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
                           'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Z'])
    
    # Look for known category letters first
    category_candidates = [e for e in elements if str(e['class']).upper() in known_categories]
    
    if category_candidates:
        # Found explicit category
        category_elements = [category_candidates[0]]  # Use the first detected category
        number_elements = [e for e in elements if e not in category_elements]
        # Sort number elements by x-coordinate
        number_elements.sort(key=lambda e: e['center'][0])
    else:
        # No explicit category detected - handle rows and positioning
        if vertical_range > 20:  # Threshold for multiple rows
            # Group elements by vertical position
            median_y = sum(y_values) / len(y_values)

            upper_row = [e for e in elements if e['center'][1] < median_y]
            lower_row = [e for e in elements if e['center'][1] >= median_y]

            # Sort each row by x-coordinate
            upper_row.sort(key=lambda e: e['center'][0])
            lower_row.sort(key=lambda e: e['center'][0])

            # For Dubai plates, typically upper row is category and lower row is number
            if len(upper_row) < len(lower_row) and len(upper_row) <= 2:
                if any(str(e['class']).isalpha() for e in upper_row):
                    category_elements = upper_row
                    number_elements = lower_row
                else:
                    # Upper row doesn't look like a category - mark as unknown
                    category_elements = []
                    number_elements = elements
                    number_elements.sort(key=lambda e: e['center'][0])
            else:
                # Look for alphabetic characters that could indicate category
                alpha_elements = [e for e in elements if isinstance(e['class'], str) and e['class'].isalpha()]
                if alpha_elements:
                    # Find the leftmost alphabetic element
                    alpha_elements.sort(key=lambda e: e['center'][0])
                    category_elements = [alpha_elements[0]]
                    number_elements = [e for e in elements if e not in category_elements]
                    number_elements.sort(key=lambda e: e['center'][0])
                else:
                    # Don't default to first element as category if it's not alphabetic
                    category_elements = []
                    number_elements = elements
                    number_elements.sort(key=lambda e: e['center'][0])
        else:
            # Single row plate - sort horizontally
            elements.sort(key=lambda e: e['center'][0])

            # Check if first element is a valid alphabetic category
            if elements and isinstance(elements[0]['class'], str) and elements[0]['class'].upper() in known_categories:
                category_elements = [elements[0]]
                number_elements = elements[1:]
            else:
                # Check for gaps between elements
                gaps = []
                gap_indices = []
                for i in range(1, len(elements)):
                    gap = elements[i]['center'][0] - elements[i-1]['center'][0]
                    gaps.append(gap)
                    gap_indices.append(i-1)

                if gaps:
                    avg_gap = sum(gaps) / len(gaps)
                    significant_gap_indices = [i for i, g in zip(gap_indices, gaps) if g > 1.5 * avg_gap]

                    if significant_gap_indices:
                        # Use the most significant gap to split category and number
                        max_gap_idx = max(significant_gap_indices, key=lambda i: gaps[i])
                        left_elements = elements[:max_gap_idx+1]
                        right_elements = elements[max_gap_idx+1:]
                        
                        # Only use left elements as category if they're alphabetic
                        if all(isinstance(e['class'], str) and e['class'].isalpha() for e in left_elements):
                            category_elements = left_elements
                            number_elements = right_elements
                        else:
                            # Left elements don't look like a category
                            category_elements = []
                            number_elements = elements
                    else:
                        # Check for leftmost element being alphabetic (common for category)
                        if isinstance(elements[0]['class'], str) and elements[0]['class'].isalpha():
                            category_elements = [elements[0]]
                            number_elements = elements[1:]
                        else:
                            category_elements = []
                            number_elements = elements
                else:
                    # Only use first element as category if it's alphabetic
                    if elements and isinstance(elements[0]['class'], str) and elements[0]['class'].isalpha():
                        category_elements = [elements[0]]
                        number_elements = elements[1:] if len(elements) > 1 else []
                    else:
                        category_elements = []
                        number_elements = elements

    # Extract category and plate number as strings
    if category_elements:
        category = ''.join([str(e['class']) for e in category_elements])
    else:
        category = 'Unknown'
    
    plate_number = ''.join([str(e['class']) for e in number_elements])

    # Post-processing
    if category == 'Unknown' and len(plate_number) > 3:
        # Try to find category letters at the beginning
        for i, char in enumerate(plate_number):
            if char.upper() in known_categories:
                # This might be a category letter
                if i == 0:  # If it's the first character
                    category = plate_number[0]
                    plate_number = plate_number[1:]
                    break

    return {
        'emirate': emirate,
        'category': category,
        'plate_number': plate_number
    }

In [5]:
def process_license_plate_image(image_path, model, conf_threshold=0.25, iou_threshold=0.45, img_size=640, save_output=False):
    """Process a license plate image: detect objects, extract license info, and visualize results"""
    # Load and process image
    start_time = time.time()

    # Check if the image exists
    if not os.path.exists(image_path):
        print(f"Error: Image path '{image_path}' does not exist")
        return None

    try:
        dataset = LoadImages(image_path, img_size=img_size)

        all_detections = []
        license_info = None
        result_img = None
        original_img = None

        for path, img, img_original, _, _ in dataset:
            # Store original image
            original_img = img_original.copy()

            # Run inference
            img = torch.tensor(img).to(model.device).float() / 255.0
            if img.ndimension() == 3:
                img = img.unsqueeze(0)

            # Get predictions
            pred = non_max_suppression(model(img), conf_threshold, iou_threshold)

            # Process detections
            annotator = Annotator(img_original.copy())

            if pred[0] is not None and len(pred[0]) > 0:
                detections = pred[0]
                # Scale boxes to original image dimensions
                detections[:, :4] = scale_boxes(img.shape[2:], detections[:, :4], img_original.shape).round()

                # Process each detection
                for *xyxy, conf, cls_id in detections:
                    cls_id = int(cls_id)
                    cls_name = model.names[cls_id] if cls_id < len(model.names) else f"Class {cls_id}"
                    confidence = float(conf)
                    bbox = [int(x) for x in xyxy]

                    # Add detection to list
                    all_detections.append({
                        'class': cls_name,
                        'confidence': confidence,
                        'bbox': bbox
                    })

                    # Draw box on image
                    label = f"{cls_name} {confidence:.2f}"
                    annotator.box_label(xyxy, label, colors(cls_id, True))

            # Extract license information
            license_info = extract_license_info(all_detections)

            # Get the annotated image
            result_img = annotator.result()

            # Save output if requested (now disabled by default)
            if save_output:
                output_dir = 'outputs'
                os.makedirs(output_dir, exist_ok=True)
                file_name = Path(path).stem
                output_path = os.path.join(output_dir, f"license_info_{file_name}.jpg")
                cv2.imwrite(output_path, result_img)
                print(f"Saved result to {output_path}")

            # Break after first image (in case of video input)
            break

        # Calculate processing time
        processing_time = time.time() - start_time

        return {
            'license_info': license_info,
            'original_img': original_img,
            'result_img': result_img,
            'detections': all_detections,
            'processing_time': processing_time
        }
    except Exception as e:
        import traceback
        print(f"Error processing image: {e}")
        traceback.print_exc()
        return None

In [6]:
def process_license_plates_batch(model, test_dir, num_images=5, conf_threshold=0.25, iou_threshold=0.45):
    """Process multiple license plate images and display results"""
    # Ensure test directory exists
    if not os.path.exists(test_dir):
        print(f"Error: Test directory '{test_dir}' does not exist")
        return None

    # Get list of image files
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = []

    for ext in image_extensions:
        image_files.extend(list(Path(test_dir).glob(f'*{ext}')))

    if not image_files:
        print(f"No images found in {test_dir}")
        return None

    # Select random images or all if fewer than requested
    import random
    if len(image_files) > num_images:
        selected_images = random.sample(image_files, num_images)
    else:
        selected_images = image_files

    print(f"Processing {len(selected_images)} images from {test_dir}")

    # Process each image
    results = []
    for img_path in selected_images:
        print(f"\nProcessing image: {img_path.name}")
        result = process_license_plate_image(
            str(img_path), model, conf_threshold, iou_threshold, save_output=False)

        if result is None:
            print(f"Failed to process image: {img_path.name}")
            continue

        results.append(result)

        # Display original and processed images side by side
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

        # Original image
        orig_img_rgb = cv2.cvtColor(result['original_img'], cv2.COLOR_BGR2RGB)
        ax1.imshow(orig_img_rgb)
        ax1.set_title("Original Image")
        ax1.axis('off')

        # Annotated image with detections
        result_img_rgb = cv2.cvtColor(result['result_img'], cv2.COLOR_BGR2RGB)
        ax2.imshow(result_img_rgb)
        ax2.set_title("Detected Objects")
        ax2.axis('off')

        plt.tight_layout()
        plt.suptitle(f"License Plate: {img_path.name}", fontsize=16)
        plt.show()

        # Display extracted license info directly under the images
        license_info = result['license_info']
        print("License Plate Information:")
        print(f"Emirate: {license_info['emirate']}")
        print(f"Category: {license_info['category']}")
        print(f"License Number: {license_info['plate_number']}")

    return results

In [7]:
def process_two_stage_detection(image_path, model, conf_threshold=0.25, iou_threshold=0.45, img_size=640):
    """Two-stage license plate detection approach"""
    import tempfile
    
    # Stage 1: Detect the license plate
    print("Stage 1: Detecting license plate...")
    dataset = LoadImages(image_path, img_size=img_size)
    
    # Load the original image
    for path, img, img_original, _, _ in dataset:
        original_img = img_original.copy()
        break
    
    # Run first stage detection (focus on finding the plate)
    plates = []
    img = torch.tensor(img).to(model.device).float() / 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    
    pred = non_max_suppression(model(img), conf_threshold, iou_threshold)
    
    # Process detections to find license plates
    annotator_original = Annotator(original_img.copy())
    
    if pred[0] is not None and len(pred[0]) > 0:
        detections = pred[0]
        detections[:, :4] = scale_boxes(img.shape[2:], detections[:, :4], original_img.shape).round()
        
        for *xyxy, conf, cls_id in detections:
            cls_id = int(cls_id)
            cls_name = model.names[cls_id] if cls_id < len(model.names) else f"Class {cls_id}"
            
            # Check if this is a license plate detection
            if 'plate' in str(cls_name).lower():
                x1, y1, x2, y2 = map(int, xyxy)
                plates.append({
                    'class': cls_name,
                    'confidence': float(conf),
                    'bbox': (x1, y1, x2, y2)
                })
                
                # Annotate the original image
                label = f"{cls_name} {float(conf):.2f}"
                annotator_original.box_label(xyxy, label, colors(cls_id, True))
    
    # If no license plates found, return early
    if not plates:
        print("No license plates detected in first stage.")
        return {
            'success': False,
            'original_img': original_img,
            'result_img': annotator_original.result(),
            'license_info': {'emirate': 'Unknown', 'category': 'Unknown', 'plate_number': 'Unknown'}
        }
    
    # Sort plates by confidence and take the highest one
    plates.sort(key=lambda x: x['confidence'], reverse=True)
    plate = plates[0]
    print(f"Found license plate with confidence {plate['confidence']:.2f}")
    
    # Stage 2: Extract the license plate (simple crop with margin)
    print("Stage 2: Extracting license plate...")
    
    # Extract plate region with some margin
    x1, y1, x2, y2 = plate['bbox']
    h, w = y2 - y1, x2 - x1
    
    # Add margins (20% on each side)
    margin_x = int(w * 0.20)
    margin_y = int(h * 0.20)
    
    # Ensure margins don't go out of bounds
    x1_margin = max(0, x1 - margin_x)
    y1_margin = max(0, y1 - margin_y)
    x2_margin = min(original_img.shape[1], x2 + margin_x)
    y2_margin = min(original_img.shape[0], y2 + margin_y)
    
    # Crop the plate region
    plate_img = original_img[y1_margin:y2_margin, x1_margin:x2_margin]
    
    # Stage 3: Run detection on the extracted plate
    print("Stage 3: Detecting text on extracted plate...")
    
    # Create a temporary file for the plate image
    plate_detections = []
    annotated_plate_img = None
    
    with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as tmp_file:
        temp_path = tmp_file.name
        cv2.imwrite(temp_path, plate_img)
    
    try:
        # Run detection on the plate image
        plate_dataset = LoadImages(temp_path, img_size=img_size)
        
        for _, plate_tensor, plate_img_original, _, _ in plate_dataset:
            plate_tensor = torch.tensor(plate_tensor).to(model.device).float() / 255.0
            if plate_tensor.ndimension() == 3:
                plate_tensor = plate_tensor.unsqueeze(0)
            
            # Run model on the plate image with slightly lower confidence threshold
            plate_pred = non_max_suppression(model(plate_tensor), conf_threshold * 0.7, iou_threshold)
            
            # Annotator for plate image
            annotator_plate = Annotator(plate_img_original.copy())
            
            if plate_pred[0] is not None and len(plate_pred[0]) > 0:
                plate_det = plate_pred[0]
                plate_det[:, :4] = scale_boxes(plate_tensor.shape[2:], plate_det[:, :4], plate_img_original.shape).round()
                
                for *xyxy, conf, cls_id in plate_det:
                    cls_id = int(cls_id)
                    cls_name = model.names[cls_id] if cls_id < len(model.names) else f"Class {cls_id}"
                    confidence = float(conf)
                    bbox = [int(x) for x in xyxy]
                    
                    # Skip 'plate' class in the second pass - we only want the characters
                    if 'plate' in str(cls_name).lower():
                        continue
                    
                    plate_detections.append({
                        'class': cls_name,
                        'confidence': confidence,
                        'bbox': bbox
                    })
                    
                    # Annotate the plate image
                    label = f"{cls_name} {confidence:.2f}"
                    annotator_plate.box_label(xyxy, label, colors(cls_id, True))
            
            # Get the annotated plate image
            annotated_plate_img = annotator_plate.result()
            break  # Only process the first image
            
    finally:
        # Clean up the temporary file
        if os.path.exists(temp_path):
            os.remove(temp_path)
    
    # Extract license info from the plate detections
    license_info = extract_license_info(plate_detections)
    
    # Prepare the final annotated original image with license info
    final_img = annotator_original.result()
    
    # Add license info text to the original image
    cv2.putText(final_img, f"Emirate: {license_info['emirate']}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(final_img, f"Category: {license_info['category']}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(final_img, f"Plate Number: {license_info['plate_number']}", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    print(f"Detection complete! Found: {license_info}")
    
    return {
        'success': True,
        'original_img': original_img,
        'result_img': final_img,
        'plate_img': plate_img,  # Cropped plate without annotations
        'annotated_plate_img': annotated_plate_img,  # Annotated plate
        'license_info': license_info,
        'plate_detections': plate_detections
    }

In [8]:
def process_two_stage_batch(model, test_dir, num_images=5, conf_threshold=0.25, iou_threshold=0.45):
    """Process multiple images using two-stage detection"""
    # Get list of image files
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = []
    
    for ext in image_extensions:
        image_files.extend(list(Path(test_dir).glob(f'*{ext}')))
    
    if not image_files:
        print(f"No images found in {test_dir}")
        return None
    
    # Select random images or all if fewer than requested
    import random
    if len(image_files) > num_images:
        selected_images = random.sample(image_files, num_images)
    else:
        selected_images = image_files
    
    print(f"Processing {len(selected_images)} images using two-stage detection...")
    
    # Process each image
    results = []
    for img_path in selected_images:
        print(f"\nProcessing image: {img_path.name}")
        result = process_two_stage_detection(str(img_path), model, conf_threshold, iou_threshold)
        
        if result is None:
            print(f"Failed to process image: {img_path.name}")
            continue
            
        results.append(result)
        
        # Display results
        fig, axs = plt.subplots(1, 3, figsize=(20, 8))
        
        # Original image
        orig_img_rgb = cv2.cvtColor(result['original_img'], cv2.COLOR_BGR2RGB)
        axs[0].imshow(orig_img_rgb)
        axs[0].set_title("Original Image")
        axs[0].axis('off')
        
        # Annotated original image
        result_img_rgb = cv2.cvtColor(result['result_img'], cv2.COLOR_BGR2RGB)
        axs[1].imshow(result_img_rgb)
        axs[1].set_title("Detected License Plate")
        axs[1].axis('off')
        
        # Cropped and annotated plate
        if result['success'] and 'annotated_plate_img' in result and result['annotated_plate_img'] is not None:
            plate_img_rgb = cv2.cvtColor(result['annotated_plate_img'], cv2.COLOR_BGR2RGB)
            axs[2].imshow(plate_img_rgb)
            axs[2].set_title("Cropped Plate with Detections")
        else:
            # If plate extraction failed, show a blank panel
            axs[2].imshow(np.ones((100, 200, 3), dtype=np.uint8) * 200)
            axs[2].set_title("Plate Extraction Failed")
        axs[2].axis('off')
        
        plt.tight_layout()
        plt.suptitle(f"License Plate: {img_path.name}", fontsize=16)
        plt.subplots_adjust(top=0.9)
        plt.show()
        
        # Display extracted license info
        license_info = result['license_info']
        print("License Plate Information:")
        print(f"Emirate: {license_info['emirate']}")
        print(f"Category: {license_info['category']}")
        print(f"License Number: {license_info['plate_number']}")
        
        # Display detection details for second stage
        if result['success'] and result['plate_detections']:
            print("\nPlate Detection Details:")
            print(f"{'Class':<15} {'Confidence':<10} {'Bounding Box (x1,y1,x2,y2)'}")
            print("-" * 60)
            for det in result['plate_detections']:
                cls = det['class']
                conf = det['confidence']
                bbox = det['bbox']
                print(f"{cls:<15} {conf:.2f}       ({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})")
    
    return results

In [9]:
def display_two_stage_results(result):
    """Display the results of two-stage license plate detection"""
    if not result['success']:
        # Show original image with message that no license plate was detected
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(result['result_img'], cv2.COLOR_BGR2RGB))
        plt.title("No license plate detected")
        plt.axis('off')
        plt.show()
        return

    # Create a figure with 2 rows and 2 columns
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))

    # Original image
    axs[0, 0].imshow(cv2.cvtColor(result['original_img'], cv2.COLOR_BGR2RGB))
    axs[0, 0].set_title("Original Image")
    axs[0, 0].axis('off')

    # Annotated original image
    axs[0, 1].imshow(cv2.cvtColor(result['result_img'], cv2.COLOR_BGR2RGB))
    axs[0, 1].set_title("Stage 1: License Plate Detection")
    axs[0, 1].axis('off')

    # Extracted plate image
    axs[1, 0].imshow(cv2.cvtColor(result['plate_img'], cv2.COLOR_BGR2RGB))
    axs[1, 0].set_title("Stage 2: Extracted Plate")
    axs[1, 0].axis('off')

    # Annotated plate image
    axs[1, 1].imshow(cv2.cvtColor(result['annotated_plate_img'], cv2.COLOR_BGR2RGB))
    axs[1, 1].set_title("Stage 3: Text Detection on Plate")
    axs[1, 1].axis('off')

    plt.tight_layout()
    plt.suptitle(f"Two-Stage License Plate Detection", fontsize=16)
    plt.show()

    # Print license information
    license_info = result['license_info']
    print("License Plate Information:")
    print(f"Emirate: {license_info['emirate']}")
    print(f"Category: {license_info['category']}")
    print(f"License Number: {license_info['plate_number']}")

In [10]:
def compare_methods(image_path, model):
    """Compare single-stage vs two-stage detection on the same image"""
    print(f"Processing image: {Path(image_path).name}")

    # Run single-stage detection
    print("\n===== SINGLE STAGE DETECTION =====")
    single_result = process_license_plate_image(image_path, model)

    # Run two-stage detection
    print("\n===== TWO STAGE DETECTION =====")
    two_stage_result = process_two_stage_detection(image_path, model)

    # Display single-stage results
    print("\nSingle-stage detection results:")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    # Original image
    orig_img_rgb = cv2.cvtColor(single_result['original_img'], cv2.COLOR_BGR2RGB)
    ax1.imshow(orig_img_rgb)
    ax1.set_title("Original Image")
    ax1.axis('off')

    # Annotated image with detections
    result_img_rgb = cv2.cvtColor(single_result['result_img'], cv2.COLOR_BGR2RGB)
    ax2.imshow(result_img_rgb)
    ax2.set_title("Single-Stage Detection")
    ax2.axis('off')

    plt.tight_layout()
    plt.suptitle(f"License Plate: {Path(image_path).name} (Single-Stage)", fontsize=16)
    plt.show()

    # Print single-stage license info
    license_info_single = single_result['license_info']
    print("License Plate Information (Single-Stage):")
    print(f"Emirate: {license_info_single['emirate']}")
    print(f"Category: {license_info_single['category']}")
    print(f"License Number: {license_info_single['plate_number']}")

    # Display two-stage results
    print("\nTwo-stage detection results:")
    display_two_stage_results(two_stage_result)

    # Return both results for comparison
    return {
        'single_stage': single_result,
        'two_stage': two_stage_result
    }

## Load the YOLOv9 Model

Now that we've defined all our functions, let's load the model and test the license plate detection.

In [11]:
# Set paths for the model and test data
weights_path = os.path.join(yolov9_path, "runs/train/gelan-c2/weights/best.pt")
test_dir = os.path.join(yolov9_path, "data/plates_dataset/test/images")

# Check if paths exist
print(f"Model path exists: {os.path.exists(weights_path)}")
print(f"Test directory exists: {os.path.exists(test_dir)}")

try:
    # Load the model
    print(f"Loading model from {weights_path}")
    model = DetectMultiBackend(weights_path)
    model.eval()
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    import traceback
    traceback.print_exc()

Model path exists: False
Test directory exists: False
Loading model from /Users/zg0ul/Coding/SAGER/dubai_license/src/yolov9/runs/train/gelan-c2/weights/best.pt
Error loading model: [Errno 2] No such file or directory: '/Users/zg0ul/Coding/SAGER/dubai_license/src/yolov9/runs/train/gelan-c2/weights/best.pt'


Traceback (most recent call last):
  File "/var/folders/wn/m2rhf88j6b54x678vc1821200000gn/T/ipykernel_27618/1698736148.py", line 12, in <module>
    model = DetectMultiBackend(weights_path)
  File "/Users/zg0ul/Coding/SAGER/dubai_license/src/yolov9/models/common.py", line 705, in __init__
    model = attempt_load(weights if isinstance(weights, list) else w, device=device, inplace=True, fuse=fuse)
  File "/Users/zg0ul/Coding/SAGER/dubai_license/src/yolov9/models/experimental.py", line 243, in attempt_load
    ckpt = torch.load(attempt_download(w), map_location='cpu')  # load
  File "/Users/zg0ul/miniconda3/envs/ai_cv_conda/lib/python3.10/site-packages/torch/serialization.py", line 1425, in load
    with _open_file_like(f, "rb") as opened_file:
  File "/Users/zg0ul/miniconda3/envs/ai_cv_conda/lib/python3.10/site-packages/torch/serialization.py", line 751, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/Users/zg0ul/miniconda3/envs/ai_cv_conda/lib/python3.10/site-pa

## Test Two-Stage Detection

Let's test the two-stage detection process on multiple images.

In [ ]:
# Run two-stage detection on a batch of images
batch_results = process_two_stage_batch(model, test_dir, num_images=3)

if batch_results:
    print(f"Successfully processed {len(batch_results)} images with two-stage detection")
else:
    print("No images were successfully processed")

## Compare Detection Methods

Let's compare the single-stage and two-stage detection methods on a sample image.

In [ ]:
# Add any additional testing code here